## Print edge_length, edge_embedding, edge_attr values and check

In [2]:
import torch
import numpy as np
edge_length_list = []
edge_embedding_list = []
edge_attr_list = []
for i in range(10):
    edge_length_list.append(torch.load(f"edge_length_{i}.pth", map_location=torch.device('cpu')))
    edge_embedding_list.append(torch.load(f"edge_embedding_{i}.pth", map_location=torch.device('cpu')))
    edge_attr_list.append(torch.load(f"edge_attr_{i}.pth", map_location=torch.device('cpu')))


In [2]:
for i in range(len(edge_length_list)):
    print(torch.equal(edge_length_list[0], edge_length_list[i]))
    print(torch.equal(edge_embedding_list[0], edge_embedding_list[i]))
    print(torch.equal(edge_attr_list[0], edge_attr_list[i]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [18]:
# filter test!!
condition = edge_length < 4
new_edge_length = edge_length[condition]
new_edge_embedding = edge_embedding[condition]
new_edge_attr = edge_attr[condition]
print(new_edge_length.shape)
print(new_edge_embedding.shape)
print(new_edge_attr.shape)

torch.Size([818])
torch.Size([818, 8])
torch.Size([818, 9])


## Naive test for edge filtering for multi cutoff

In [1]:
import torch
import numpy as np
import sevenn._keys as KEY
import sevenn.util as util
from sevenn.atom_graph_data import AtomGraphData
from sevenn.nn.sequential import AtomGraphSequential
from sevenn.train.dataload import unlabeled_atoms_to_graph
from ase.io import read
from sevenn.nn.edge_embedding import (
    BesselBasis,
    EdgeEmbedding,
    PolynomialCutoff,
    SphericalEncoding,
    XPLORCutoff,
)
from sevenn._const import DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG
from sevenn.model_build import init_edge_embedding

In [2]:
atoms = read('Mg4Ta8O24.cif')
graph_6cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 6.0))
graph_4cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 4.0))

In [3]:
# there's only edge_vec and edge_index. There's no edge_length, edge_embed, and edge_attr. -> we should build Edgeembedding layer!
for key, value in graph_6cut.items():
    print(key)
    try:
        print(graph_6cut[key].shape, graph_4cut[key].shape)
    except:
        pass
    # print('---------------------------')

x
torch.Size([36]) torch.Size([36])
edge_index
torch.Size([2, 2910]) torch.Size([2, 818])
pos
torch.Size([36, 3]) torch.Size([36, 3])
node_attr
torch.Size([36]) torch.Size([36])
atomic_numbers
torch.Size([36]) torch.Size([36])
edge_vec
torch.Size([2910, 3]) torch.Size([818, 3])
cell_lattice_vectors
torch.Size([3, 3]) torch.Size([3, 3])
pbc_shift
torch.Size([2910, 3]) torch.Size([818, 3])
cell_volume
torch.Size([]) torch.Size([])
num_atoms
torch.Size([]) torch.Size([])
data_info


In [4]:
# make 2 edge_embedding layer!!
import copy
graph_6cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 6.0))
graph_4cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 4.0))
model_cfg1 = copy.deepcopy(DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG)
model_cfg2 = copy.deepcopy(DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG)
model_cfg3 = copy.deepcopy(DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG)
model_cfg4 = copy.deepcopy(DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG)
model_cfg1['cutoff_function'] = {'cutoff_function_name': 'poly_cut'}
model_cfg1['cutoff'] = 6
model_cfg2['cutoff_function'] = {'cutoff_function_name': 'poly_cut'}
model_cfg2['cutoff'] = 4
print(model_cfg1)
print(model_cfg2)
edge_embedding_layer_6cut = init_edge_embedding(model_cfg1)
edge_embedding_layer_4cut = init_edge_embedding(model_cfg2)
graph_6cut_from_6data_poly = edge_embedding_layer_6cut(copy.deepcopy(graph_6cut))
graph_4cut_from_4data_poly = edge_embedding_layer_4cut(copy.deepcopy(graph_4cut))
# how about mixed?
graph_4cut_from_6data_poly = edge_embedding_layer_4cut(copy.deepcopy(graph_6cut))
graph_6cut_from_4data_poly = edge_embedding_layer_6cut(copy.deepcopy(graph_4cut))

# new edge_embedding layer!!
model_cfg3['cutoff_function'] = {'cutoff_function_name': 'XPLOR', 'cutoff_on': 3.5}
model_cfg3['cutoff'] = 4
edge_embedding_layer_4cut_xplor = init_edge_embedding(model_cfg3)
model_cfg4['cutoff_function'] = {'cutoff_function_name': 'XPLOR', 'cutoff_on': 5.5}
model_cfg4['cutoff'] = 6
edge_embedding_layer_6cut_xplor = init_edge_embedding(model_cfg4)
print(model_cfg3)
print(model_cfg4)

graph_4cut_from_4data_xplor = edge_embedding_layer_4cut_xplor(copy.deepcopy(graph_4cut))
graph_6cut_from_6data_xplor = edge_embedding_layer_6cut_xplor(copy.deepcopy(graph_4cut))
graph_4cut_from_6data_xplor = edge_embedding_layer_4cut_xplor(copy.deepcopy(graph_4cut))
graph_6cut_from_4data_xplor = edge_embedding_layer_6cut_xplor(copy.deepcopy(graph_4cut))

{'irreps_manual': False, 'channel': 32, 'lmax': 1, 'lmax_edge': -1, 'lmax_node': -1, 'is_parity': True, 'radial_basis': {'radial_basis_name': 'bessel'}, 'cutoff_function': {'cutoff_function_name': 'poly_cut'}, 'act_radial': 'silu', 'cutoff': 6, 'weight_nn_hidden_neurons': [64, 64], 'num_convolution_layer': 3, 'act_scalar': {'e': 'silu', 'o': 'tanh'}, 'act_gate': {'e': 'silu', 'o': 'tanh'}, 'conv_denominator': 'avg_num_neigh', 'train_denominator': False, 'train_shift_scale': False, 'use_bias_in_linear': False, 'readout_as_fcn': False, 'readout_fcn_hidden_neurons': [30, 30], 'readout_fcn_activation': 'relu', 'self_connection_type': 'nequip', 'interaction_type': 'nequip', '_normalize_sph': True}
{'irreps_manual': False, 'channel': 32, 'lmax': 1, 'lmax_edge': -1, 'lmax_node': -1, 'is_parity': True, 'radial_basis': {'radial_basis_name': 'bessel'}, 'cutoff_function': {'cutoff_function_name': 'poly_cut'}, 'act_radial': 'silu', 'cutoff': 4, 'weight_nn_hidden_neurons': [64, 64], 'num_convolutio

In [5]:
check_key = [KEY.EDGE_EMBEDDING, KEY.EDGE_LENGTH, KEY.EDGE_ATTR]
for key in check_key:
    # check just 4cut vs 6cut from 4cut data
    print(f'{key}')
    print(f'4cut vs 6cut from 4cut data: {torch.equal(graph_4cut_from_4data_poly[key], graph_6cut_from_4data_poly[key])}')
    # 6cut vs 4cut from 6cut data
    print(f'6cut vs 4cut from 6cut data: {torch.equal(graph_6cut_from_6data_poly[key], graph_4cut_from_6data_poly[key])}')
    print('-----------------------------------')

edge_embedding
4cut vs 6cut from 4cut data: False
6cut vs 4cut from 6cut data: False
-----------------------------------
edge_length
4cut vs 6cut from 4cut data: True
6cut vs 4cut from 6cut data: True
-----------------------------------
edge_attr
4cut vs 6cut from 4cut data: True
6cut vs 4cut from 6cut data: True
-----------------------------------


In [6]:

check_key = [KEY.EDGE_EMBEDDING, KEY.EDGE_LENGTH, KEY.EDGE_ATTR]
for key in check_key:
    # check just 4cut vs 6cut from 4cut data
    print(f'{key}')
    print(f'4cut vs 4cut: {torch.equal(graph_4cut_from_4data_poly[key], graph_4cut_from_4data_xplor[key])}')
    # 6cut vs 4cut from 6cut data
    print(f'6cut vs 6cut: {torch.equal(graph_6cut_from_6data_poly[key], graph_6cut_from_6data_xplor[key])}')
    print(f'4cut from 6: {torch.equal(graph_4cut_from_6data_poly[key], graph_4cut_from_6data_xplor[key])}')
    print(f'6cut from 4: {torch.equal(graph_6cut_from_4data_poly[key], graph_6cut_from_4data_xplor[key])}')


edge_embedding
4cut vs 4cut: False
6cut vs 6cut: False
4cut from 6: False
6cut from 4: False
edge_length
4cut vs 4cut: True
6cut vs 6cut: False
4cut from 6: False
6cut from 4: True
edge_attr
4cut vs 4cut: True
6cut vs 6cut: False
4cut from 6: False
6cut from 4: True


## Does filtering work??

In [ ]:
# compare 4cut 4data and 4cut 6data
edge_embed_4cut_4data = graph_4cut_from_4data_poly[KEY.EDGE_EMBEDDING].detach().numpy()
edge_embed_4cut_6data = graph_4cut_from_6data_poly[KEY.EDGE_EMBEDDING].detach().numpy()
edge_length_4cut_4data = graph_4cut_from_4data_poly[KEY.EDGE_LENGTH].numpy()
edge_length_4cut_6data = graph_4cut_from_6data_poly[KEY.EDGE_LENGTH].numpy()
edge_attr_4cut_4data = graph_4cut_from_4data_poly[KEY.EDGE_ATTR].numpy()
edge_attr_4cut_6data = graph_4cut_from_6data_poly[KEY.EDGE_ATTR].numpy()

# filter edge_length < 4
condition = edge_length_4cut_6data <= 4
new_edge_length_4cut_6data = edge_length_4cut_6data[condition]
new_edge_embed_4cut_6data = edge_embed_4cut_6data[condition]
new_edge_attr_4cut_6data = edge_attr_4cut_6data[condition]

# check if they are same
sorted_indices_4cut_4data = np.lexsort(
    (edge_embed_4cut_4data[:, 2], edge_embed_4cut_4data[:, 1], edge_embed_4cut_4data[:, 0])
)
sorted_indices_4cut_6data = np.lexsort(
    (new_edge_embed_4cut_6data[:, 2], new_edge_embed_4cut_6data[:, 1], new_edge_embed_4cut_6data[:, 0])
)
print(np.allclose(edge_embed_4cut_4data[sorted_indices_4cut_4data], new_edge_embed_4cut_6data[sorted_indices_4cut_6data]))
print(np.allclose(edge_length_4cut_4data[sorted_indices_4cut_4data], new_edge_length_4cut_6data[sorted_indices_4cut_6data]))
print(np.allclose(edge_attr_4cut_4data[sorted_indices_4cut_4data], new_edge_attr_4cut_6data[sorted_indices_4cut_6data]))


True
True
True


In [93]:
# compare 4cut 4data and 4cut 6data
edge_embed_4cut_4data = graph_4cut_from_4data_xplor[KEY.EDGE_EMBEDDING].detach().numpy()
edge_embed_4cut_6data = graph_4cut_from_6data_xplor[KEY.EDGE_EMBEDDING].detach().numpy()
edge_length_4cut_4data = graph_4cut_from_4data_xplor[KEY.EDGE_LENGTH].numpy()
edge_length_4cut_6data = graph_4cut_from_6data_xplor[KEY.EDGE_LENGTH].numpy()
edge_attr_4cut_4data = graph_4cut_from_4data_xplor[KEY.EDGE_ATTR].numpy()
edge_attr_4cut_6data = graph_4cut_from_6data_xplor[KEY.EDGE_ATTR].numpy()

# filter edge_length < 4
condition = edge_length_4cut_6data <= 4
new_edge_length_4cut_6data = edge_length_4cut_6data[condition]
new_edge_embed_4cut_6data = edge_embed_4cut_6data[condition]
new_edge_attr_4cut_6data = edge_attr_4cut_6data[condition]

# check if they are same
sorted_indices_4cut_4data = np.lexsort(
    (edge_embed_4cut_4data[:, 2], edge_embed_4cut_4data[:, 1], edge_embed_4cut_4data[:, 0])
)
sorted_indices_4cut_6data = np.lexsort(
    (new_edge_embed_4cut_6data[:, 2], new_edge_embed_4cut_6data[:, 1], new_edge_embed_4cut_6data[:, 0])
)
print(np.allclose(edge_embed_4cut_4data[sorted_indices_4cut_4data], new_edge_embed_4cut_6data[sorted_indices_4cut_6data]))
print(np.allclose(edge_length_4cut_4data[sorted_indices_4cut_4data], new_edge_length_4cut_6data[sorted_indices_4cut_6data]))
print(np.allclose(edge_attr_4cut_4data[sorted_indices_4cut_4data], new_edge_attr_4cut_6data[sorted_indices_4cut_6data]))


True
True
True


## multicutoff model making

In [1]:
import torch
import numpy as np
import sevenn._keys as KEY
import sevenn.util as util
from sevenn.atom_graph_data import AtomGraphData
from sevenn.nn.sequential import AtomGraphSequential
from sevenn.train.dataload import unlabeled_atoms_to_graph
from ase.io import read
from sevenn.nn.edge_embedding import (
    BesselBasis,
    EdgeEmbedding,
    PolynomialCutoff,
    SphericalEncoding,
    XPLORCutoff,
)
from sevenn._const import DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG, model_defaults
from sevenn.model_build import init_edge_embedding, build_E3_equivariant_model

atoms = read('Mg4Ta8O24.cif')
graph_6cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 6.0))
graph_4cut = AtomGraphData.from_numpy_dict(unlabeled_atoms_to_graph(atoms, 4.0))

In [2]:
import copy
model_cfg_multi = copy.deepcopy(DEFAULT_E3_EQUIVARIANT_MODEL_CONFIG)

model_cfg_multi['cutoff_function'] = {'cutoff_function_name': 'poly_cut'}
model_cfg_multi['cutoff'] = 5
model_cfg_multi['multi_cutoff'] = [5, 5, 4, 5, 5]


multi_edge_embedding_layer_6cut = init_edge_embedding(model_cfg_multi)

multi_graph_6cut_from_6data_poly = multi_edge_embedding_layer_6cut(copy.deepcopy(graph_6cut))


In [3]:
import sys
print(multi_graph_6cut_from_6data_poly)
print(sys.getsizeof(multi_graph_6cut_from_6data_poly))


AtomGraphData(
  x=[36],
  edge_index=[2, 2910],
  pos=[36, 3],
  node_attr=[36],
  atomic_numbers=[36],
  edge_vec=[2910, 3],
  cell_lattice_vectors=[3, 3],
  pbc_shift=[2910, 3],
  cell_volume=410.892333984375,
  num_atoms=36,
  data_info={},
  edge_length=[2910],
  edge_embedding4=[818, 8],
  edge_attr4=[818, 4],
  edge_index4=[2, 818],
  edge_embedding5=[1488, 8],
  edge_attr5=[1488, 4],
  edge_index5=[2, 1488]
)
48


## model build test

In [44]:
from typing import Any, Dict, List, Tuple, Union
import warnings
def _patch_old_config(config: Dict[str, Any]):
    # Fixing my old mistakes
    if config[KEY.CUTOFF_FUNCTION][KEY.CUTOFF_FUNCTION_NAME] == 'XPLOR':
        config[KEY.CUTOFF_FUNCTION].pop('poly_cut_p_value', None)
    if KEY.TRAIN_DENOMINTAOR not in config:
        config[KEY.TRAIN_DENOMINTAOR] = config.pop('train_avg_num_neigh', False)
    _opt = config.pop('optimize_by_reduce', None)
    if _opt is False:
        raise ValueError(
            'This checkpoint(optimize_by_reduce: False) is no longer supported'
        )
    if KEY.CONV_DENOMINATOR not in config:
        config[KEY.CONV_DENOMINATOR] = 0.0
    if KEY._NORMALIZE_SPH not in config:
        config[KEY._NORMALIZE_SPH] = False
        # Warn this in the docs, not here for SevenNet-0 (22May2024)
    return config

def _map_old_model(old_model_state_dict):
    """
    For compatibility with old namings (before 'correct' branch merged 2404XX)
    Map old model's module names to new model's module names
    """
    _old_module_name_mapping = {
        'EdgeEmbedding': 'edge_embedding',
        'reducing nn input to hidden': 'reduce_input_to_hidden',
        'reducing nn hidden to energy': 'reduce_hidden_to_energy',
        'rescale atomic energy': 'rescale_atomic_energy',
    }
    for i in range(10):
        _old_module_name_mapping[f'{i} self connection intro'] = (
            f'{i}_self_connection_intro'
        )
        _old_module_name_mapping[f'{i} convolution'] = f'{i}_convolution'
        _old_module_name_mapping[f'{i} self interaction 2'] = (
            f'{i}_self_interaction_2'
        )
        _old_module_name_mapping[f'{i} equivariant gate'] = f'{i}_equivariant_gate'

    new_model_state_dict = {}
    for k, v in old_model_state_dict.items():
        key_name = k.split('.')[0]
        follower = '.'.join(k.split('.')[1:])
        if 'denumerator' in follower:
            follower = follower.replace('denumerator', 'denominator')
        if key_name in _old_module_name_mapping:
            new_key_name = _old_module_name_mapping[key_name] + '.' + follower
            new_model_state_dict[new_key_name] = v
        else:
            new_model_state_dict[k] = v
    return new_model_state_dict


In [50]:
# load weight manually
checkpoint = util.pretrained_name_to_path('7net-0')
check_point_loaded = torch.load(checkpoint, map_location='cpu', weights_only=False)
model_state_dict = check_point_loaded['model_state_dict']
config = check_point_loaded['config']
defaults = {**model_defaults(config)}
config = _patch_old_config(config)

for k, v in defaults.items():
    if k not in config:
        warnings.warn(f'{k} not in config, using default value {v}', UserWarning)
        config[k] = v
for k, v in config.items():
    if isinstance(v, torch.Tensor):
        config[k] = v.cpu()

# multi cutoff model
multi_cutoff_list = [5, 5, 4, 5, 5]
print(f'multi cutoff list: {multi_cutoff_list}')
# config['multi_cutoff'] = multi_cutoff_list
model = build_E3_equivariant_model(config)

missing, _ = model.load_state_dict(model_state_dict, strict=False)
if len(missing) > 0:
    updated = _map_old_model(model_state_dict)
    missing, not_used = model.load_state_dict(updated, strict=False)
    if len(not_used) > 0:
        warnings.warn(f'Some keys are not used: {not_used}', UserWarning)

multi cutoff list: [5, 5, 4, 5, 5]


In [51]:
try:
    print('model mc')
    print(model.edge_embedding.basis_functions[0].coeffs)
    print(model.edge_embedding.basis_functions[1].coeffs)
    print(model._modules['0_self_interaction_1'].linear.weight)
except:
    print(model.edge_embedding.basis_function.coeffs)
    print(model._modules['0_self_interaction_1'].linear.weight)

model mc
Parameter containing:
tensor([0.5079, 1.1099, 1.7265, 2.3767, 3.0392, 3.7201, 4.4142, 5.1186],
       requires_grad=True)
Parameter containing:
tensor([-1.2098, -2.5352, -1.1502,  ...,  3.7544,  1.4783, -0.2325],
       requires_grad=True)


In [ ]:
# model mc
# Parameter containing:
# tensor([0.7854, 1.5708, 2.3562, 3.1416, 3.9270, 4.7124, 5.4978, 6.2832],
#        requires_grad=True)
# Parameter containing:
# tensor([0.6283, 1.2566, 1.8850, 2.5133, 3.1416, 3.7699, 4.3982, 5.0265],
#        requires_grad=True)
# Parameter containing:
# tensor([-1.2098, -2.5352, -1.1502,  ...,  3.7544,  1.4783, -0.2325],
#        requires_grad=True)

Parameter containing:
tensor([ 1.7966,  0.3282,  0.6795,  ..., -0.2949, -0.0243, -0.6929],
       requires_grad=True)